## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [ ]:
def get_width(image):
    return image.shape[1]

def get_height(image):
    return image.shape[0]

def get_imageSize_for_cv2(image):
    return (get_width(image), get_height(image))

class Pix2Meter:
    def __init__(self, ym_per_pix, xm_per_pix):
        # meters per pixel in y dimension
        self.ym_per_pix = ym_per_pix
        # meters per pixel in x dimension
        self.xm_per_pix = xm_per_pix
    
    def y_pix_2_meter(self, pix):
        return pix * self.ym_per_pix

    def x_pix_2_meter(self, pix):
        return pix * self.xm_per_pix

def create_default_pix2Meter():
    return Pix2Meter(ym_per_pix = 30/720, xm_per_pix = 3.7/700)

def savefig(image):
    plt.savefig(image, bbox_inches = 'tight')

## Compute the camera calibration matrix and distortion coefficients given a set of chessboard images

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

def get_objpoints_imgpoints(draw):
    patternSize = (9, 6)
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((np.prod(patternSize), 3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, patternSize, None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            if draw:
                img = cv2.drawChessboardCorners(img, patternSize, corners, ret)
                cv2.imshow('img', img)
                cv2.waitKey(500)

    cv2.destroyAllWindows()
    return objpoints, imgpoints

In [ ]:
import pickle
import os

def calibrateCamera():
    objpoints, imgpoints = get_objpoints_imgpoints(draw = False)
    img = cv2.imread('camera_cal/calibration1.jpg')
    # Do camera calibration given object points and image points
    _, cameraMatrix, distCoeffs, _, _ = cv2.calibrateCamera(objpoints,
                                                            imgpoints,
                                                            get_imageSize_for_cv2(img),
                                                            None,
                                                            None)
    calibration_data = {"cameraMatrix": cameraMatrix, "distCoeffs": distCoeffs}
    return calibration_data

def get_calibrateCamera_pickle_file():
    return 'camera_cal/wide_dist_pickle.p'

# Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
def save_calibration_data(calibration_data):
    pickle.dump(calibration_data, open(get_calibrateCamera_pickle_file(), "wb"))

# Read in the saved camera matrix and distortion coefficients
def load_calibration_data():
    calibration_data = pickle.load(open(get_calibrateCamera_pickle_file(), "rb"))
    return calibration_data

def get_calibration_data():
    if not os.path.isfile(get_calibrateCamera_pickle_file()):
        save_calibration_data(calibrateCamera())
    return load_calibration_data()

## Apply a distortion correction to raw images

adapted from https://github.com/udacity/CarND-Camera-Calibration/blob/master/camera_calibration.ipynb

In [ ]:
def undistort(image, calibration_data):
    return cv2.undistort(image,
                         calibration_data['cameraMatrix'],
                         calibration_data['distCoeffs'],
                         None,
                         calibration_data['cameraMatrix'])

In [ ]:
%matplotlib inline

def visualize_undistortion(original_image, undistorted_image):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    ax1.imshow(original_image)
    ax1.set_title('Original Image', fontsize=30)
    ax2.imshow(undistorted_image)
    ax2.set_title('Undistorted Image', fontsize=30)

In [ ]:
original_image = cv2.imread('camera_cal/calibration1.jpg')
undistorted_image = undistort(original_image, get_calibration_data())
# cv2.imwrite('output_images/calibration1_undistorted.png', dst)

visualize_undistortion(original_image, undistorted_image)
savefig('output_images/calibration1_undistorted.png')

In [ ]:
original_image = cv2.cvtColor(cv2.imread('test_images/test1.jpg'), cv2.COLOR_BGR2RGB)
undistorted_image = undistort(original_image, get_calibration_data())

visualize_undistortion(original_image, undistorted_image)
savefig('output_images/test1_undistorted.png')

## Use color transforms, gradients, etc., to create a thresholded binary image

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def apply_thresholds(image, thresholds):
    result = np.zeros_like(image)
    result[(image >= thresholds[0]) & (image <= thresholds[1])] = 1
    return result

    
def detect_edges_using_x_derivative(image, sx_thresh):
    # Sobel x
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    return apply_thresholds(scaled_sobel, sx_thresh)


def combine_with_or(binary_img1, binary_img2):
    combined_binary = np.zeros_like(binary_img1)
    combined_binary[(binary_img1 == 1) | (binary_img2 == 1)] = 1
    return combined_binary


def create_color_image(green_image, blue_image):
    red_image = np.zeros_like(green_image)
    return np.dstack((red_image, green_image, blue_image)) * 255
    

def get_l_channel_and_s_channel(image):
    # Convert to HLS color space and separate the S and L channels
    hls = cv2.cvtColor(np.copy(image), cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    return l_channel, s_channel
    

def create_binary_images(image, s_channel_thresholds, x_derivative_thresholds):        
    l_channel, s_channel = get_l_channel_and_s_channel(image)    
    x_derivative_binary = detect_edges_using_x_derivative(l_channel, x_derivative_thresholds)
    s_channel_binary = apply_thresholds(s_channel, s_channel_thresholds)
    
    return (create_color_image(green_image = x_derivative_binary, blue_image = s_channel_binary),
            combine_with_or(x_derivative_binary, s_channel_binary))

In [ ]:
image = mpimg.imread('test_images/test2.jpg')
color_binary, combined_binary = create_binary_images(image,
                                                     s_channel_thresholds = (170, 255),
                                                     x_derivative_thresholds = (60, 100))

# Plot the result
f, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(48, 18))
f.tight_layout()

ax1.imshow(image)
ax1.set_title('Original Image')

ax2.imshow(color_binary)
ax2.set_title('color_binary')

ax3.imshow(combined_binary, cmap = 'gray')
ax3.set_title('combined_binary')

savefig('output_images/test1_binary.png')

In [ ]:
image = mpimg.imread('test_images/tree_with_shadow.jpg')
color_binary, combined_binary = create_binary_images(image,
                                                     s_channel_thresholds = (170, 255),
                                                     x_derivative_thresholds = (60, 100))

# Plot the result
f, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(48, 18))
f.tight_layout()

ax1.imshow(image)
ax1.set_title('Original Image')

ax2.imshow(color_binary)
ax2.set_title('color_binary')

ax3.imshow(combined_binary, cmap = 'gray')
ax3.set_title('combined_binary')

## Apply a perspective transform to rectify binary image ("birds-eye view")

In [ ]:
def get_img_src_dst():
    img = mpimg.imread('test_images/straight_lines2.jpg')
    src = np.float32(
        [
            (get_width(img)/2 - 60, get_height(img)/2 + 100),
            (get_width(img)/6 + 5, get_height(img)),
            (get_width(img)*5/6 + 45, get_height(img)),
            (get_width(img)/2 + 65, get_height(img)/2 + 100)
        ])
    dst = np.float32(
        [
            (get_width(img)/4, 0),
            (get_width(img)/4, get_height(img)),
            (get_width(img)*3/4, get_height(img)),
            (get_width(img)*3/4, 0)
        ])
    return img, src, dst
    
# TODO: cache the result of this function!!! It is always the same result
def getPerspectiveTransform():
    _, src, dst = get_img_src_dst()
    return cv2.getPerspectiveTransform(src, dst)

def warpPerspective(image):
    return cv2.warpPerspective(src = image,
                               M = getPerspectiveTransform(),
                               dsize = get_imageSize_for_cv2(image),
                               flags = cv2.INTER_NEAREST)

In [ ]:
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Read in an image
img, src, dst = get_img_src_dst()
print("src:\n", src)
print("dst:\n", dst)

undist = undistort(img, get_calibration_data())

def polylines(pts, image):
    cv2.polylines(image, [pts.astype(int).reshape((-1, 1, 2))], True, (255, 0, 0), 6)

undist_with_source_points = np.copy(undist)
polylines(src, undist_with_source_points)

warped = warpPerspective(undist)
polylines(dst, warped)

f, (ax1, ax2) = plt.subplots(1, 2, figsize = (24, 9))
f.tight_layout()
ax1.imshow(undist_with_source_points)
ax1.set_title('Undistorted image with source points drawn', fontsize = 25)
ax2.imshow(warped)
ax2.set_title('Warped result with destination points drawn', fontsize = 25)
plt.subplots_adjust(left = 0., right = 1, top = 0.9, bottom = 0.)

savefig('output_images/test2_warped_straight_lines.png')

## Detect lane pixels and fit to find the lane boundary

In [ ]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2

def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[get_height(binary_warped)//2:,:], axis = 0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(get_height(binary_warped)//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = get_height(binary_warped) - (window+1)*window_height
        win_y_high = get_height(binary_warped) - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,
                      (win_xleft_low, win_y_low),
                      (win_xleft_high, win_y_high),
                      (0, 255, 0),
                      2) 
        cv2.rectangle(out_img,
                      (win_xright_low, win_y_low),
                      (win_xright_high, win_y_high),
                      (0, 255, 0),
                      2) 
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) &
                          (nonzeroy < win_y_high) & 
                          (nonzerox >= win_xleft_low) &
                          (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) &
                           (nonzeroy < win_y_high) & 
                           (nonzerox >= win_xright_low) &
                           (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def polyfit_line(y, x, pix2Meter):
    coeffs = np.polyfit(pix2Meter.y_pix_2_meter(y), pix2Meter.x_pix_2_meter(x), 2) if len(y) != 0 else [0, 0, 0]
    return np.poly1d(coeffs)


def polyfit_left_line_right_line(lane_pixels, height, pix2Meter):
    leftx, lefty, rightx, righty, _ = lane_pixels
    left_line = polyfit_line(lefty, leftx, pix2Meter)
    right_line = polyfit_line(righty, rightx, pix2Meter)
    y = pix2Meter.y_pix_2_meter(np.linspace(0, height-1, height))
    return left_line, right_line, y


def draw_line(x, y, image):
    pts = np.dstack((x.astype(int), y.astype(int)))[0]
    pts = pts.reshape((-1, 1, 2))
    cv2.polylines(image, [pts], False, (255, 255, 0), 3)


def fit_polynomial(lane_pixels, get_polyfitted_left_line_right_line_in_pixel):
    leftx, lefty, rightx, righty, out_img = lane_pixels
    left_line, right_line, y = get_polyfitted_left_line_right_line_in_pixel()

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    # Plots the left and right polynomials on the lane lines
    draw_line(left_line(y), y, out_img)
    draw_line(right_line(y), y, out_img)
    return out_img, left_line(y), right_line(y), y

## Determine the curvature of the lane and vehicle position with respect to center

In [ ]:
def R_curve(line, y):
    return ((1 + (2*line.coef[0]*y + line.coef[1])**2)**1.5) / np.absolute(2*line.coef[0])

def measure_curvature_real(get_polyfitted_left_line_right_line_in_meter):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    left_line, right_line, y = get_polyfitted_left_line_right_line_in_meter()

    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(y)
    
    # Calculation of R_curve (radius of curvature)
    return R_curve(left_line, y_eval), R_curve(right_line, y_eval)

def get_radius_of_curvature(get_polyfitted_left_line_right_line_in_meter):
    left_curverad, right_curverad = measure_curvature_real(get_polyfitted_left_line_right_line_in_meter)
    return left_curverad, right_curverad, (left_curverad + right_curverad)/2

In [ ]:
def get_vehicle_position(image, get_polyfitted_left_line_right_line_in_meter):
    left_line, right_line, y = get_polyfitted_left_line_right_line_in_meter()

    def get_midpoint_of_lane():
        y_eval = np.max(y)
        return (left_line(y_eval) + right_line(y_eval))/2

    def get_center_of_car():
        return create_default_pix2Meter().x_pix_2_meter(get_width(image)/2)

    return get_center_of_car() - get_midpoint_of_lane()

## Warp the detected lane boundaries back onto the original image

In [ ]:
# Create an image to draw the lines on
def project_lane_area_onto_undistorted_image(warped, undist, Minv, lane_pixels, get_polyfitted_left_line_right_line_in_pixel):
    out_img, left_fitx, right_fitx, ploty = fit_polynomial(lane_pixels, get_polyfitted_left_line_right_line_in_pixel)

    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, get_imageSize_for_cv2(image))
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    return result, out_img

## Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position

In [ ]:
def putText(image, text, bottomLeftCornerOfText):
    cv2.putText(img = image,
                text = text,
                org = bottomLeftCornerOfText,
                fontFace = cv2.FONT_HERSHEY_SIMPLEX,
                fontScale = 1.5,
                color = (255, 255, 255),
                thickness = 2)

def printInfos(image, curverad, vehicle_position):
    radius_of_curvature_text = "Radius of Curvature = {:.2f} km".format(curverad/1000)
    putText(image, radius_of_curvature_text, (50, 50))
    
    vehicle_position_text = "Vehicle is {:.0f} cm {} of center".format(
        abs(vehicle_position*100),
        'right' if vehicle_position>0 else 'left')
    putText(image, vehicle_position_text, (50, 100))

## Run Pipeline

In [ ]:
def polyfit_left_line_right_line_helper(results, pix2Meter):
    return polyfit_left_line_right_line(lane_pixels = results[PipelineResultType.LANE_PIXELS],
                                        height = get_height(results[PipelineResultType.PERSPECTIVE_COMBINED_BINARY]),
                                        pix2Meter = pix2Meter)
    
    
def get_polyfitted_left_line_right_line_in_pixel(results):
    left_line, right_line, y = polyfit_left_line_right_line_helper(results, Pix2Meter(ym_per_pix = 1, xm_per_pix = 1))
    return lambda: (left_line, right_line, y)

def get_polyfitted_left_line_right_line_in_meter(results):
    left_line, right_line, y = polyfit_left_line_right_line_helper(results, create_default_pix2Meter())
    return lambda: (left_line, right_line, y)

In [ ]:
from enum import Enum

class PipelineResultType(Enum):
    INPUT = 'INPUT'
    UNDISTORTED = 'UNDISTORTED'
    COLOR_BINARY = 'COLOR_BINARY'
    COMBINED_BINARY = 'COMBINED_BINARY'
    LANE_PIXELS = 'LANE_PIXELS'
    LEFT_LINE = 'LEFT_LINE'
    RIGHT_LINE = 'RIGHT_LINE'
    PERSPECTIVE_COMBINED_BINARY = 'PERSPECTIVE_COMBINED_BINARY'
    PERSPECTIVE_UNDISTORTED = 'PERSPECTIVE_UNDISTORTED'
    LINES_WITH_SLIDING_WINDOWS = 'LINES_WITH_SLIDING_WINDOWS'
    RADIUS_OF_CURVATURE = 'RADIUS_OF_CURVATURE'
    LEFT_RADIUS_OF_CURVATURE = 'LEFT_RADIUS_OF_CURVATURE'
    RIGHT_RADIUS_OF_CURVATURE = 'RIGHT_RADIUS_OF_CURVATURE'
    LINES = 'LINES'
    OUTPUT = 'OUTPUT'

In [ ]:
class PipelineResults:
    
    def __init__(self):
        self.results = {}

    def getImageResults(self):
        return {entity: self.results[entity] for entity in self.results.keys() & {PipelineResultType.INPUT,
                                                                                  PipelineResultType.UNDISTORTED,
                                                                                  PipelineResultType.COLOR_BINARY,
                                                                                  PipelineResultType.COMBINED_BINARY,
                                                                                  PipelineResultType.PERSPECTIVE_COMBINED_BINARY,
                                                                                  PipelineResultType.PERSPECTIVE_UNDISTORTED,
                                                                                  PipelineResultType.LINES_WITH_SLIDING_WINDOWS,
                                                                                  PipelineResultType.LINES,
                                                                                  PipelineResultType.OUTPUT}}

In [ ]:
class Line():
    
    def __init__(self,
                 polynomial_coeffs_in_pixel,
                 averaged_polynomial_coeffs_in_pixel,
                 
                 polynomial_coeffs_in_meter,
                 averaged_polynomial_coeffs_in_meter):
        
        self.polynomial_coeffs_in_pixel = polynomial_coeffs_in_pixel
        self.averaged_polynomial_coeffs_in_pixel = averaged_polynomial_coeffs_in_pixel
        
        self.polynomial_coeffs_in_meter = polynomial_coeffs_in_meter
        self.averaged_polynomial_coeffs_in_meter = averaged_polynomial_coeffs_in_meter

In [ ]:
from numpy.linalg import inv
from collections import deque

# TODO: add outlier detection and reuse previous frame or average of previous frames if outlier frame detected
# TODO: use weighted average?
# TODO: im Allgemeinen können Koeffizienten in Pixel immer in Meter umgerechnet werden, deshalb muß man immer nur Koeffizienten in einer einzigen Einheit speichern. Bsp: left_polynomial_coeffs_in_pixel und left_polynomial_coeffs_in_meter
class Pipeline:
    
    def __init__(self, average_n_frames):
        create_deque = lambda: deque(maxlen = average_n_frames)
        self.left_polynomial_coeffs_in_pixel = create_deque()
        self.right_polynomial_coeffs_in_pixel = create_deque()
        self.left_polynomial_coeffs_in_meter = create_deque()
        self.right_polynomial_coeffs_in_meter = create_deque()

    def process_image(self, image):
        pipelineResults = PipelineResults()
        pipelineResults.results = {PipelineResultType.INPUT: image}
        calibration_data = get_calibration_data()

        pipelineResults.results[PipelineResultType.UNDISTORTED] = undistort(image, calibration_data)

        pipelineResults.results[PipelineResultType.COLOR_BINARY], pipelineResults.results[PipelineResultType.COMBINED_BINARY] = \
            create_binary_images(pipelineResults.results[PipelineResultType.UNDISTORTED],
                                 s_channel_thresholds = (170, 255),
                                 x_derivative_thresholds = (60, 100))

        pipelineResults.results[PipelineResultType.PERSPECTIVE_COMBINED_BINARY] = warpPerspective(pipelineResults.results[PipelineResultType.COMBINED_BINARY])
        pipelineResults.results[PipelineResultType.PERSPECTIVE_UNDISTORTED] = warpPerspective(pipelineResults.results[PipelineResultType.UNDISTORTED])

        pipelineResults.results[PipelineResultType.LANE_PIXELS] = find_lane_pixels(pipelineResults.results[PipelineResultType.PERSPECTIVE_COMBINED_BINARY])
        
        left_line_pixel, right_line_pixel, _ = get_polyfitted_left_line_right_line_in_pixel(pipelineResults.results)()
        self.left_polynomial_coeffs_in_pixel.append(left_line_pixel.coef)
        self.right_polynomial_coeffs_in_pixel.append(right_line_pixel.coef)
            
        left_line_meter, right_line_meter, _ = get_polyfitted_left_line_right_line_in_meter(pipelineResults.results)()
        self.left_polynomial_coeffs_in_meter.append(left_line_meter.coef)
        self.right_polynomial_coeffs_in_meter.append(right_line_meter.coef)
        
        pipelineResults.results[PipelineResultType.LEFT_LINE] =\
            Line(polynomial_coeffs_in_pixel = left_line_pixel.coef,
                 averaged_polynomial_coeffs_in_pixel = self.__average(self.left_polynomial_coeffs_in_pixel),
                 polynomial_coeffs_in_meter = left_line_meter.coef,
                 averaged_polynomial_coeffs_in_meter = self.__average(self.left_polynomial_coeffs_in_meter))
        
        pipelineResults.results[PipelineResultType.RIGHT_LINE] =\
            Line(polynomial_coeffs_in_pixel = right_line_pixel.coef,
                 averaged_polynomial_coeffs_in_pixel = self.__average(self.right_polynomial_coeffs_in_pixel),
                 polynomial_coeffs_in_meter = right_line_meter.coef,
                 averaged_polynomial_coeffs_in_meter = self.__average(self.right_polynomial_coeffs_in_meter))
        
        image, pipelineResults.results[PipelineResultType.LINES_WITH_SLIDING_WINDOWS] = \
            project_lane_area_onto_undistorted_image(pipelineResults.results[PipelineResultType.PERSPECTIVE_COMBINED_BINARY],
                                                     pipelineResults.results[PipelineResultType.UNDISTORTED],
                                                     inv(getPerspectiveTransform()),
                                                     pipelineResults.results[PipelineResultType.LANE_PIXELS],
                                                     self.__get_averaged_polyfitted_left_line_right_line_in_pixel(pipelineResults.results))
        pipelineResults.results[PipelineResultType.LINES] = np.copy(image)
        
        (pipelineResults.results[PipelineResultType.LEFT_RADIUS_OF_CURVATURE],
         pipelineResults.results[PipelineResultType.RIGHT_RADIUS_OF_CURVATURE],
         pipelineResults.results[PipelineResultType.RADIUS_OF_CURVATURE]) =\
            get_radius_of_curvature(self.__get_averaged_polyfitted_left_line_right_line_in_meter(pipelineResults.results))

        printInfos(image,
                   curverad = pipelineResults.results[PipelineResultType.RADIUS_OF_CURVATURE],
                   vehicle_position = get_vehicle_position(pipelineResults.results[PipelineResultType.PERSPECTIVE_COMBINED_BINARY],
                                                           self.__get_averaged_polyfitted_left_line_right_line_in_meter(pipelineResults.results)))
        pipelineResults.results[PipelineResultType.OUTPUT] = image
        return pipelineResults

    def __get_averaged_polyfitted_left_line_right_line_in_meter(self, results):
        _, _, y = get_polyfitted_left_line_right_line_in_meter(results)()
        left_line = np.poly1d(results[PipelineResultType.LEFT_LINE].averaged_polynomial_coeffs_in_meter)
        right_line  = np.poly1d(results[PipelineResultType.RIGHT_LINE].averaged_polynomial_coeffs_in_meter)
        return lambda: (left_line, right_line, y)

    def __get_averaged_polyfitted_left_line_right_line_in_pixel(self, results):
        _, _, y = get_polyfitted_left_line_right_line_in_pixel(results)()
        left_line = np.poly1d(results[PipelineResultType.LEFT_LINE].averaged_polynomial_coeffs_in_pixel)
        right_line  = np.poly1d(results[PipelineResultType.RIGHT_LINE].averaged_polynomial_coeffs_in_pixel)
        return lambda: (left_line, right_line, y)
    
    def __average(self, coeffs):
        return np.average(coeffs, axis = 0)

In [ ]:
import numpy as np

from enum import Enum

class Turn(Enum):
    LEFT = 'LEFT'
    RIGHT = 'RIGHT'

class OutlierDetector:
    
    def isOutlier(self, left_polynomial_coeffs, right_polynomial_coeffs):
        return self.__get_turn(left_polynomial_coeffs) != self.__get_turn(right_polynomial_coeffs)
    
    def __get_turn(self, coeffs):
        return Turn.LEFT if np.sign(coeffs[0]) <= 0 else Turn.RIGHT

In [ ]:
import unittest

class OutlierDetectorTestCase(unittest.TestCase):
    
    def test_detect_outlier1(self):
        # GIVEN
        # 'test_images/tree_with_shadow.jpg'
        outlierDetector = OutlierDetector()
        left_polynomial_coeffs = [-5.56730030e-04, 2.84663144e-01, 6.70790697e+01]
        right_polynomial_coeffs = [ 1.72166506e-03, -1.01784168e+00, 1.06929226e+03]
        
        # WHEN
        is_outlier = outlierDetector.isOutlier(left_polynomial_coeffs, right_polynomial_coeffs)
        
        # THEN
        self.assertTrue(is_outlier)
        
    def test_detect_outlier2(self):
        # GIVEN
        # 'test_images/test2.jpg'
        outlierDetector = OutlierDetector()
        left_polynomial_coeffs =  [-2.21828741e-04, 2.90135493e-01, 2.76016161e+02]
        right_polynomial_coeffs = [-3.24329524e-06, 2.09897766e-01, 8.99956004e+02]
        
        # WHEN
        is_outlier = outlierDetector.isOutlier(left_polynomial_coeffs, right_polynomial_coeffs)
        
        # THEN
        self.assertFalse(is_outlier)
        

suite = unittest.TestLoader().loadTestsFromTestCase(OutlierDetectorTestCase)
runner = unittest.TextTestRunner(verbosity = 2)
runner.run(suite)

In [ ]:
import unittest

class PipelineTestCase(unittest.TestCase):
    
    left_coeffs1_pixel = [7.706599689821455e-05, -0.12631282250945297, 404.84301374593497]
    right_coeffs1_pixel = [0.00022263474036021452, -0.2500912700122073, 1084.947412992218]
    left_coeffs2_pixel = [0.00015721580019633635, -0.3385262518591035, 509.20927542320135]
    right_coeffs2_pixel = [0.00019224195369502113, -0.2806320109024004, 1119.5589021553017]

    def test_average_frames(self):
        # GIVEN
        pipeline = Pipeline(average_n_frames = 2)
        
        def average(coeffs1, coeffs2):
            return [(a+b)/2 for a, b in zip(coeffs1, coeffs2)]

        # WHEN & THEN
        self.__process_image_and_check_coeffs(pipeline,
                                              'test_images/test4.jpg',
                                              left_coeffs_pixel_expected = self.left_coeffs1_pixel,
                                              left_coeffs_meter_expected = self.__transform_to_meter(self.left_coeffs1_pixel),
                                              left_coeffs_avg_pixel_expected = self.left_coeffs1_pixel,
                                              right_coeffs_pixel_expected = self.right_coeffs1_pixel,
                                              right_coeffs_avg_pixel_expected = self.right_coeffs1_pixel)
        
        left_coeffs2_avg_pixel = average(self.left_coeffs1_pixel, self.left_coeffs2_pixel)
        right_coeffs2_avg_pixel = average(self.right_coeffs1_pixel, self.right_coeffs2_pixel)
        results = self.__process_image_and_check_coeffs(pipeline,
                                                        'test_images/test6.jpg',
                                                        left_coeffs_pixel_expected = self.left_coeffs2_pixel,
                                                        left_coeffs_avg_pixel_expected = left_coeffs2_avg_pixel,
                                                        right_coeffs_pixel_expected = self.right_coeffs2_pixel,
                                                        right_coeffs_avg_pixel_expected = right_coeffs2_avg_pixel)
        
        # TODO: refactor, extract method
        # THEN radius of curvature is based on averaged polynomial coefficients of left line and right line.
        y_eval = np.max(get_polyfitted_left_line_right_line_in_meter(results)()[2])
        r_curve = (R_curve(np.poly1d(self.__transform_to_meter(left_coeffs2_avg_pixel)), y_eval) + R_curve(np.poly1d(self.__transform_to_meter(right_coeffs2_avg_pixel)), y_eval))/2
        np.testing.assert_almost_equal(actual = results[PipelineResultType.RADIUS_OF_CURVATURE],
                                       desired = r_curve,
                                       decimal = 7)

    def __transform_to_meter(self, coeffs):
        pix2Meter = create_default_pix2Meter()
        return [coeffs[0]*pix2Meter.xm_per_pix/(pix2Meter.ym_per_pix**2),
                coeffs[1]*pix2Meter.xm_per_pix/pix2Meter.ym_per_pix,
                coeffs[2]*pix2Meter.xm_per_pix]

    def __process_image_and_check_coeffs(self,
                                         pipeline,
                                         image,
                                         left_coeffs_pixel_expected = None,
                                         left_coeffs_meter_expected = None,
                                         left_coeffs_avg_pixel_expected = None,
                                         right_coeffs_pixel_expected = None,
                                         right_coeffs_avg_pixel_expected = None):
        # GIVEN
        image = mpimg.imread(image)

        # WHEN
        results = pipeline.process_image(image).results

        # THEN
        if left_coeffs_pixel_expected is not None:
            self.assertEqual(left_coeffs_pixel_expected, list(results[PipelineResultType.LEFT_LINE].polynomial_coeffs_in_pixel))
        if left_coeffs_meter_expected is not None:
            np.testing.assert_almost_equal(actual = list(results[PipelineResultType.LEFT_LINE].polynomial_coeffs_in_meter),
                                           desired = left_coeffs_meter_expected,
                                           decimal = 7)
        if left_coeffs_avg_pixel_expected is not None:
            self.assertEqual(left_coeffs_avg_pixel_expected, list(results[PipelineResultType.LEFT_LINE].averaged_polynomial_coeffs_in_pixel))
        if right_coeffs_pixel_expected is not None:
            self.assertEqual(right_coeffs_pixel_expected, list(results[PipelineResultType.RIGHT_LINE].polynomial_coeffs_in_pixel))
        if right_coeffs_avg_pixel_expected is not None:
            self.assertEqual(right_coeffs_avg_pixel_expected, list(results[PipelineResultType.RIGHT_LINE].averaged_polynomial_coeffs_in_pixel))

        return results

    def test_average_n_frames(self):
        # GIVEN
        pipeline = Pipeline(average_n_frames = 1)
                    
        # WHEN & THEN
        self.__process_image_and_check_coeffs(pipeline,
                                              'test_images/test4.jpg',
                                              left_coeffs_avg_pixel_expected = self.left_coeffs1_pixel,
                                              right_coeffs_avg_pixel_expected = self.right_coeffs1_pixel)
        
        self.__process_image_and_check_coeffs(pipeline,
                                              'test_images/test6.jpg',
                                              left_coeffs_avg_pixel_expected = self.left_coeffs2_pixel,
                                              right_coeffs_avg_pixel_expected = self.right_coeffs2_pixel)
        

suite = unittest.TestLoader().loadTestsFromTestCase(PipelineTestCase)
runner = unittest.TextTestRunner(verbosity = 2)
runner.run(suite)

In [ ]:
def pipeline(image):
    return Pipeline(average_n_frames = 1).process_image(image)

In [ ]:
import unittest
import numpy as np

class SanityCheckTestCase(unittest.TestCase):
    
    def __init__(self, *args, **kwargs):
        super(SanityCheckTestCase, self).__init__(*args, **kwargs)
        self.images = [#'test_images/straight_lines1.jpg',
                       #'test_images/straight_lines2.jpg',
                       'test_images/test1.jpg',
                       #'test_images/test2.jpg',
                       #'test_images/test3.jpg',
                       #'test_images/test4.jpg',
                       #'test_images/test5.jpg',
                       #'test_images/test6.jpg',
                       #'test_images/tree_with_shadow.jpg'
        ]

    def get_lane_widths(self, results):
        left_line, right_line, y = polyfit_left_line_right_line(lane_pixels = results[PipelineResultType.LANE_PIXELS],
                                                                height = get_height(results[PipelineResultType.PERSPECTIVE_COMBINED_BINARY]),
                                                                pix2Meter = create_default_pix2Meter())
        lane_widths = right_line(y) - left_line(y)
        return y, lane_widths

        
    def test_lane_width_is_3_point_7(self):
        for image in self.images:
            with self.subTest(image = image):
                self.__test_lane_width_is_3_point_7(image)

    def __test_lane_width_is_3_point_7(self, image):
        # GIVEN
        image = mpimg.imread(image)

        # WHEN
        results = pipeline(image).results

        # THEN
        lane_widths = self.get_lane_widths(results)[1]
        np.testing.assert_almost_equal(actual = lane_widths,
                                       desired = [3.7] * len(lane_widths),
                                       decimal = 2)
        
    def test_lane_has_constant_width(self):
        for image in self.images:
            with self.subTest(image = image):
                self.__test_lane_has_constant_width(image)

    def __test_lane_has_constant_width(self, image):
        # GIVEN
        image = mpimg.imread(image)
        
        # WHEN
        results = pipeline(image)
        
        # THEN
        lane_widths = self.get_lane_widths(results)[1]
        # TODO: or test var(lane_widths) almost zero?
        np.testing.assert_almost_equal(actual = lane_widths,
                                       desired = [lane_widths[0]] * len(lane_widths),
                                       decimal = 2)


suite = unittest.TestLoader().loadTestsFromTestCase(SanityCheckTestCase)
runner = unittest.TextTestRunner(verbosity = 2)
runner.run(suite)

In [ ]:
def run_pipeline_draw_save(input_image, ouput_image_pattern):
    pipelineResults = pipeline(mpimg.imread(input_image))
    for resultType in pipelineResults.getImageResults():
        plt.figure(figsize = (10, 10))
        plt.imshow(pipelineResults.results[resultType], cmap = 'gray')
        savefig(ouput_image_pattern.format(resultType.name))

In [ ]:
run_pipeline_draw_save(input_image = 'test_images/test1.jpg',
                       ouput_image_pattern = 'output_images/test1_{}.png')

In [ ]:
run_pipeline_draw_save(input_image = 'test_images/test2.jpg',
                       ouput_image_pattern = 'output_images/test2_{}.png')

In [ ]:
run_pipeline_draw_save(input_image = 'test_images/tree_with_shadow.jpg',
                       ouput_image_pattern = 'output_images/tree_with_shadow_{}.png')

In [ ]:
def draw(input_image, output_image):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize = (24, 9))
    f.tight_layout()

    ax1.imshow(input_image)
    ax1.set_title('image -> pipeline', fontsize = 50)

    ax2.imshow(output_image, cmap = 'gray')
    ax2.set_title('pipeline -> image', fontsize = 50)

    plt.subplots_adjust(left = 0., right = 1, top = 0.9, bottom = 0.)
        
def run_pipeline_and_draw(image):
    input_image = mpimg.imread(image)
    # image = cv2.imread('test_images/straight_lines2.jpg')
    draw(input_image = input_image,
         output_image = pipeline(input_image).results[PipelineResultType.OUTPUT])

In [ ]:
run_pipeline_and_draw('test_images/straight_lines1.jpg')
run_pipeline_and_draw('test_images/straight_lines2.jpg')
run_pipeline_and_draw('test_images/test1.jpg')
run_pipeline_and_draw('test_images/test2.jpg')
run_pipeline_and_draw('test_images/test3.jpg')
run_pipeline_and_draw('test_images/test4.jpg')
run_pipeline_and_draw('test_images/test5.jpg')
run_pipeline_and_draw('test_images/test6.jpg')
run_pipeline_and_draw('test_images/tree_with_shadow.jpg')

## Video

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def create_image_func():
    pipeline = Pipeline(average_n_frames = 50)
    return lambda image: pipeline.process_image(image).results[PipelineResultType.OUTPUT]

In [ ]:
def process_video(input_video, output_video):
    clip = VideoFileClip(input_video)
    clip_processed = clip.fl_image(create_image_func())
    %time clip_processed.write_videofile(output_video, audio = False)

In [ ]:
def embed_video(video):
    return HTML("""
    <video width="960" height="540" controls>
      <source src="{0}">
    </video>
    """.format(video))

In [ ]:
output_video = 'test_videos_output/project_video.mp4'
#process_video(input_video = 'test_videos/project_video.mp4', output_video = output_video)
embed_video(output_video)

In [ ]:
output_video = 'test_videos_output/challenge_video.mp4'
#process_video(input_video = 'test_videos/challenge_video.mp4', output_video = output_video)
embed_video(output_video)

In [ ]:
output_video = 'test_videos_output/harder_challenge_video.mp4'
#process_video(input_video = 'test_videos/harder_challenge_video.mp4', output_video = output_video)
embed_video(output_video)